<a href="https://colab.research.google.com/github/lahirukavinda/ADL-AI-Summit---Pre-Hackathon/blob/main/ADLHack_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Principal Component Analysis with up-sampling

Tested for MLP

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import random

df = pd.read_csv('/content/drive/MyDrive/Projects/ADLHackothon/train_dataset.csv', error_bad_lines=False).fillna(0)
df.shape[0]

10500

In [ ]:
df.head()

,primary_identifier,device_type,device_category,gender,district_name,age_group,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,dusage_max,dusage_avg,dusage_days,dusage_stddev,vusage_onnet_sum,vusage_onnet_max,vusage_onnet_min,vusage_onnet_days,vusage_onnet_avg,vusage_onnet_stddev,vusage_offnet_sum,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count,next_month_plan
0,6875768,4G,Smartphone,MALE,Kegalle,40-50,219,761.62,6317.0,2.773958,622.685848,210.579939,30.0,197.116822,0.0,0.0,0.0,0.0,0.0,0.0,677.0,152.866667,0.0,27.0,25.099383,34.759221,0.0,1.0,0.0,0.0,PKG2
1,6406277,4G,Smartphone,MALE,Galle,40-50,36,2482.52,28365.0,3.079438,2569.429138,945.531235,30.0,573.299639,0.0,0.0,0.0,0.0,0.0,0.0,76.0,35.433333,0.0,15.0,5.071111,6.779644,0.0,0.0,600.0,2.0,PKG1
2,3563570,4G,Smartphone,MALE,Kandy,30-40,186,1565.55,18581.0,48.193713,2739.515326,619.378810,30.0,619.750500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,1.0,PKG6
3,1853666,4G,Smartphone,FEMALE,Colombo,40-50,24,2397.49,91071.0,22.437842,7523.355693,3035.708609,30.0,2174.203127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,1110.0,2.0,PKG4
4,2794331,4G,Smartphone,FEMALE,Kandy,50-60,37,2501.75,19736.0,53.302092,1239.220170,657.889802,30.0,329.152516,0.0,0.0,0.0,0.0,0.0,0.0,258.0,87.616667,0.0,23.0,11.256522,16.224071,0.0,1.0,1350.0,2.0,PKG4


In [ ]:
x = df.drop('next_month_plan', axis='columns')

x = x.drop('primary_identifier', axis='columns')
x = x.drop('device_type', axis='columns')
x = x.drop('device_category', axis='columns')
x = x.drop('age_group', axis='columns')

y = pd.DataFrame()
y['next_month_plan'] = df['next_month_plan'].str.replace("PKG",'').astype(int)
y.head()

,next_month_plan
0,2
1,1
2,6
3,4
4,4


In [ ]:
from sklearn.preprocessing import LabelEncoder

x["gender"] = LabelEncoder().fit_transform(x["gender"].astype(str))
x["district_name"] = LabelEncoder().fit_transform(x["district_name"].astype(str))

x.head()

,gender,district_name,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,dusage_max,dusage_avg,dusage_days,dusage_stddev,vusage_onnet_sum,vusage_onnet_max,vusage_onnet_min,vusage_onnet_days,vusage_onnet_avg,vusage_onnet_stddev,vusage_offnet_sum,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count
0,2,11,219,761.62,6317.0,2.773958,622.685848,210.579939,30.0,197.116822,0.0,0.0,0.0,0.0,0.0,0.0,677.0,152.866667,0.0,27.0,25.099383,34.759221,0.0,1.0,0.0,0.0
1,2,5,36,2482.52,28365.0,3.079438,2569.429138,945.531235,30.0,573.299639,0.0,0.0,0.0,0.0,0.0,0.0,76.0,35.433333,0.0,15.0,5.071111,6.779644,0.0,0.0,600.0,2.0
2,2,10,186,1565.55,18581.0,48.193713,2739.515326,619.378810,30.0,619.750500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,1.0
3,1,4,24,2397.49,91071.0,22.437842,7523.355693,3035.708609,30.0,2174.203127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,1110.0,2.0
4,1,10,37,2501.75,19736.0,53.302092,1239.220170,657.889802,30.0,329.152516,0.0,0.0,0.0,0.0,0.0,0.0,258.0,87.616667,0.0,23.0,11.256522,16.224071,0.0,1.0,1350.0,2.0


In [ ]:
from sklearn.preprocessing import StandardScaler

x_df_test_nom = StandardScaler().fit_transform(x)

In [ ]:
# Principal Component Analysis

from sklearn.decomposition import PCA

pca = PCA(n_components=20)

principalComponents = pca.fit_transform(x_df_test_nom)

principalDf = pd.DataFrame(data = principalComponents)
principalDf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,5.970848,1.489915,7.868198,-0.838813,-1.761466,-0.615707,-1.132850,-1.845564,-0.581755,0.022381,0.116592,0.139080,0.127858,1.563107,-0.126311,0.808223,0.069488,0.611176,1.440599,2.124498
1,-0.516941,1.519086,1.085966,-0.266180,0.266603,0.082089,-0.130233,-0.081488,-0.035880,-1.087668,-0.016617,-0.884259,-0.127966,-0.463100,0.104029,-0.133452,-0.671426,0.281392,0.665152,0.026657
2,-1.552873,-0.020535,-0.781858,-0.079568,-0.947136,-0.921522,0.755401,-0.218337,-1.004597,0.036037,0.369548,-0.378098,0.308538,0.773595,-0.132226,-0.078855,0.123984,0.164974,-0.068451,0.231978
3,-4.998287,6.366615,-0.952294,-2.664124,0.604546,-0.025080,-0.071946,0.330825,0.501154,0.768599,-0.041471,-2.155448,1.082036,0.304449,-0.037688,0.439133,-2.183503,1.265200,-0.256132,0.270757
4,1.734402,1.955793,3.664125,0.375829,0.937915,1.392855,-1.533386,-1.289440,0.606828,0.832730,0.219550,-0.170158,0.041877,-0.331654,0.168592,0.941911,-0.440200,0.294234,1.400770,0.631850


In [ ]:
# Class balancing
# Data points for each package: 3673, 1354, 1717, 735, 1558, 740, 364, 359

NUM_ENTRIES_EACH = 350
NUM_COMPONENTS = 8

def store_test_data_downsample(x, y) :
  x_df = pd.DataFrame().reindex(columns=x.columns)
  y_df = pd.DataFrame().reindex(columns=y.columns)
  for i in range(1, NUM_COMPONENTS+1):
    x_filtered = x[y['next_month_plan']==i]
    #print(len(x_filtered.index))

    x_df = x_df.append(x_filtered.head(NUM_ENTRIES_EACH), ignore_index = True)
    y_df = y_df.append(pd.DataFrame([i for j in range(NUM_ENTRIES_EACH)], columns=y.columns), ignore_index = True).astype(int)

  return x_df, y_df

def store_test_data_upsample(x, y) :
  x_df = pd.DataFrame().reindex(columns=x.columns)
  y_df = pd.DataFrame().reindex(columns=y.columns)

  mult = [3, 9, 7, 17, 8, 17, 34, 34]
  for i in range(1, NUM_COMPONENTS+1):
    x_filtered = x[y['next_month_plan']==i]
    #print(len(x_filtered.index))
    
    for j in range(0, mult[i-1]) :
      x_df = x_df.append(x_filtered, ignore_index = True)
      y_df = y_df.append(pd.DataFrame([i for j in range(len(x_filtered.index))], columns=y.columns), ignore_index = True).astype(int)

  return x_df, y_df

In [ ]:
from sklearn.model_selection import train_test_split

# principalDf = x
x_train_o, x_test_o, y_train_o, y_test_o = train_test_split(principalDf, y, random_state = 0) 
x_train_o.shape[0]

7875

In [ ]:
x_df_test, y_df_test = store_test_data_upsample(x_train_o, y_train_o)

print(y_df_test.shape[0])
print(y_df_test)

72974
       next_month_plan
0                    1
1                    1
2                    1
3                    1
4                    1
...                ...
72969                8
72970                8
72971                8
72972                8
72973                8

[72974 rows x 1 columns]


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# x_train, x_test, y_train, y_test = train_test_split(x_synth, y_synth, random_state = 0) 

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# model_xg = xgb.XGBClassifier()

# model_xg.fit(x_df_test, y_df_test)
# #model_xg.fit(x_train, y_train)
# print('Prediction Score : ', model_xg.score(x_test_o, y_test_o))
# xg_predictions = model_xg.predict(x_test_o)
# cm_xg = confusion_matrix(y_test_o, xg_predictions)
# print('Confusion matrix :\n', cm_xg)
# print('Clasification Report :\n', classification_report(y_test_o, xg_predictions))

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

model_mlp = MLPClassifier(max_iter=500, activation='relu', hidden_layer_sizes=(1000,8))
print('Model MLP:\n', model_mlp)

model_mlp.fit(x_df_test, y_df_test)
#model_mlp.fit(x_train, y_train)
print('Prediction Score of MLP: ', model_mlp.score(x_test_o, y_test_o))
mlp_predictions = model_mlp.predict(x_test_o)
cm_mlp = confusion_matrix(y_test_o, mlp_predictions)
print('Confusion matrix of MLP:\n', cm_mlp)
print('Clasification Report of MLP:\n',classification_report(y_test_o, mlp_predictions))

Model MLP:
 MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(1000, 8), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Prediction Score of MLP:  0.42742857142857144
Confusion matrix of MLP:
 [[519 271 102  16  18   6   5   0]
 [133 120  40  26  10   5   8   0]
 [ 87  84 155  30  28   6   6   2]
 [ 26  28  72  27  17   9   5   3]
 [ 18  19  28   7 217  65  20   7]
 [  7   8  12   3  78  56  25  14]
 [  2   7   4   3  15  28  13  17]
 [  5   5   5   5  15  24  14  15]]
Clasification Report of MLP:
               precision    recall  f1-score   support

           1       0.65      0.55      0.60       937
           2       0.22      0.35      0.27       342
           3       0.37      0.39      0.38       398
           4       0.23      0.14      0.18       187
           5       0.55      0.57      0.56       381
           6       0.28      0.28      0.28       203
           7       0.14      0.15      0.14        89
           8       0.26      0.17      0.21        88

    accuracy                           0.43      2625
   macro avg       0.34      0.33      0.33      2625
weighted avg       0.